In [21]:
import asyncio
import pybotters
import pandas as pd
from loguru import logger

import json
import requests
import pandas as pd
import datetime
import time
import math

In [22]:
# e.g. https://api.cryptowat.ch/markets/:exchange/:pair/ohlc
def generate_path(
        exchange: str,
        pair: str,
        min: int,
        unixTime: int) -> str:
    
    return f'https://api.cryptowat.ch/markets/{exchange}/{pair}/ohlc?periods={min}&after={unixTime}'

# print(generate_path(1675368565))

In [23]:
def str_to_timestamp(s: str) -> int:

    # str → datetime e.g.'2022-05-27 10:11:23'
    dt = datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S")

    # datetime → timestamp
    ts = datetime.datetime.timestamp(dt)

    return math.floor(ts)

ts = str_to_timestamp('2022-05-27 10:11:23')
print(ts)

1653613883.0


In [24]:
async def main(candle: int, date: str, size: int) -> pd.DataFrame:
    async with pybotters.Client() as client:
        url="https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc?periods="
        candle=candle*60
        period=("%s"%(candle))
        period=[period]
        beforeurl="&before="
        afterurl="&after="
        # The maximum size of data is 6000 depending on a timing (sometimes you get less than 6000)
        before = str_to_timestamp(date)
        after = before - (candle * size)
        URL=("%s%s%s%s%s%s"%(url,candle,afterurl,after,beforeurl,before))
        # e.g https://api.cryptowat.ch/markets/bitflyer/btcjpy/ohlc?periods=86400&after=1483196400

        response = await client.get(URL)
        body = await response.json()
        res = body['result']
        
        data = []
        for i in period:
            row = res[i]
            for column in row:
                if column[4] != 0:
                    column = column[0:6]
                    data.append(column)

        date = [price[0] for price in data]
        op = [int(price[1]) for price in data]
        hi = [int(price[2]) for price in data]
        lo = [int(price[3]) for price in data]
        cl = [int(price[4]) for price in data]
        date_datetime = map(datetime.datetime.fromtimestamp, date)
        dti = pd.DatetimeIndex(date_datetime)
        df = pd.DataFrame({"op" : op, "hi" : hi, "lo": lo, "cl" : cl}, index=dti)
        
        return df

In [25]:
if __name__ == '__main__':
    try:
        # df = await main(15, 1675368565, 6000)
        df = await main(15, '2023-02-02 20:09:25', 6000)
        print(df)
    except KeyboardInterrupt:
        pass

KeyError: 'result'